In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import os
os.chdir("/")
os.chdir("/content/drive/My Drive/Colab Notebooks/Graduation Project/")
print(os.getcwd())

/content/drive/My Drive/Colab Notebooks/Graduation Project


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import ST_GCN.data_utils as DTool
from ST_GCN.model import Model, st_gcn
from ST_GCN.gcn import ConvTemporalGraphical
from ST_GCN.graph import Graph

# 1, Prepare data

In [0]:
def loading_content(path):
    f = open(path)
    return f.readlines()

def process_file_content(file_content, path):
    res = np.zeros((6, len(file_content) , 23 , 1), float)
    for i_frame in range(139,len(file_content)):
        vector = file_content[i_frame].split()
        vector = [float(i) for i in vector]
        i_chanel = 0
        i_node = 0
        for i in range(len(vector)):
            res[i_chanel][i_frame-139][i_node][0] = vector[i]
            i_chanel += 1
            i_chanel %= 6
            if (i % 6 == 5):
                i_node += 1

    return res

def loading_data (path):
    dataset = DTool.get_dataset(path)
    path = DTool.get_motion_paths_and_labels(dataset)
    X = []
    Y = []

    for index in range(len(path[0])):
        label = path[1][index]
        file_content = loading_content(path[0][index])
        x_min = process_file_content(file_content, path[0][index])
        X.append(x_min)
        Y.append(label)

    return (X , Y)

In [0]:
data_path = "/content/drive/My Drive/Share - Rubik AI/CuttingFile"
(X_train, Y_train) = loading_data(data_path)

In [45]:
num_samples = len(X_train)
print(len(X_train))
print(type(X_train))
print(len(Y_train))
print(type(Y_train))
print(X_train[135].shape)
print(Y_train)
print(len(np.unique(Y_train)))

210
<class 'list'>
210
<class 'list'>
(6, 815, 23, 1)
[0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 8, 8, 8, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 16, 16, 16, 17, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 30, 30, 31, 31, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37]
38


In [46]:
Y_train = tf.keras.utils.to_categorical(Y_train,
                                        num_classes=None,
                                        dtype='float32'
                                       )
print(Y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


# 2, Forward

In [47]:
BATCH_SIZE = 1
NUM_EPOCHS = 20
N_BATCHES = int(num_samples/ BATCH_SIZE)
print(N_BATCHES)

num_class = 38
kernel_size = (9, 3)
in_channels = 6

210


## Create A, edge_importance, kernel_size

In [48]:
graph = Graph(layout='customer settings',
              strategy='spatial',
              max_hop=1,
              dilation=1)
# ----------------
# ADJACENCY MATRIX
# ----------------
A = graph.A
A = tf.constant(A, dtype='float32', name='A')
print(A)

# ------------------------------
#   MASK MATRIX: uncmt to choose
# ------------------------------
# # If you don't use edge_importance matrix
# edge_importance = tf.ones([10, 3, 23, 23])
# print(edge_importance)

# If you use edge_importance matrix
edge_importance = []
for i in range(10):      # WARNING: IF YOU CHANGE THE # LAYERS, YOU MUST CHANGE THIS NUMBER
  mask = tf.Variable(tf.ones_like(A))
  edge_importance.append(mask)

Tensor("A_1:0", shape=(3, 23, 23), dtype=float32)


## Build model

In [0]:
# st_Gcn networks
st_gcn1 = st_gcn(in_channels, 64, kernel_size, stride=1, dropout=0)
st_gcn2 = st_gcn(64, 64, kernel_size, stride=1, dropout=0)
st_gcn3 = st_gcn(64, 64, kernel_size, stride=1, dropout=0)
st_gcn4 = st_gcn(64, 64, kernel_size, stride=1, dropout=0)
st_gcn5 = st_gcn(64, 128, kernel_size, stride=2, dropout=0)
st_gcn6 = st_gcn(128, 128, kernel_size, stride=1, dropout=0)
st_gcn7 = st_gcn(128, 128, kernel_size, stride=1, dropout=0)
st_gcn8 = st_gcn(128, 256, kernel_size, stride=2, dropout=0)
st_gcn9 = st_gcn(256, 256, kernel_size, stride=1, dropout=0)
st_gcn10 = st_gcn(256, 256, kernel_size, stride=1, dropout=0)

# Batch Normalization
data_bn = tf.keras.layers.BatchNormalization(axis=1)

# Fully connected
fcn = tf.keras.layers.Conv2D(num_class, kernel_size=(1,1), data_format='channels_first')

## Forward

In [50]:
x_train, y_train = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 6, None, 23, 1], name='x_train'), tf.placeholder(tf.float32, shape=[BATCH_SIZE, num_class], name='y_train')
print(x_train)
print(y_train)

# dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).repeat()

# iteration = dataset.make_initializable_iterator()
# x_train, y_train = iteration.get_next()


# ---------------------------
# Error: at tf.reshape => you have to change T in func tf.reshape into -1
# ---------------------------
N, C, T, V, M = x_train.get_shape()
x = tf.transpose(x_train, perm=[0, 4, 3, 1, 2])
print(x)
x = tf.reshape(x, [N*M, V*C, -1], name=None)
print(x)
x = data_bn(x)
x = tf.reshape(x, [N, M, V, C, -1], name=None)
x = tf.transpose(x, perm=[0, 1, 3, 4, 2])
x = tf.reshape(x, [N*M, C, -1, V], name=None)

print(x)


# -------------------------------
# Forward through st_gcn networks
# -------------------------------

x, _ = st_gcn1.forward(x, A*edge_importance[0])
x, _ = st_gcn2.forward(x, A*edge_importance[1])
x, _ = st_gcn3.forward(x, A*edge_importance[2])
x, _ = st_gcn4.forward(x, A*edge_importance[3])
x, _ = st_gcn5.forward(x, A*edge_importance[4])
x, _ = st_gcn6.forward(x, A*edge_importance[5])
x, _ = st_gcn7.forward(x, A*edge_importance[6])
x, _ = st_gcn8.forward(x, A*edge_importance[7])
x, _ = st_gcn9.forward(x, A*edge_importance[8])
x, _ = st_gcn10.forward(x, A*edge_importance[9])
print(x)

# Global average pooling
x = tf.keras.layers.GlobalAveragePooling2D(data_format='channels_first')(x)
x = tf.reshape(x, [N, M, -1, 1, 1], name=None)
x = tf.reduce_mean(x, axis=1)
print(x)

# Prediction and return logits
x = fcn(x)
logits = tf.reshape(x, [x.get_shape()[0], -1], name=None)

print(logits)

Tensor("x_train_1:0", shape=(1, 6, ?, 23, 1), dtype=float32)
Tensor("y_train_1:0", shape=(1, 38), dtype=float32)
Tensor("transpose_2:0", shape=(1, 1, 23, 6, ?), dtype=float32)
Tensor("Reshape_15:0", shape=(1, 138, ?), dtype=float32)
Tensor("Reshape_17:0", shape=(1, 6, ?, 23), dtype=float32)
Tensor("Reshape_17:0", shape=(1, 6, ?, 23), dtype=float32)
Tensor("re_lu_21/Relu:0", shape=(1, 64, ?, 23), dtype=float32)
Tensor("re_lu_23/Relu:0", shape=(1, 64, ?, 23), dtype=float32)
Tensor("re_lu_25/Relu:0", shape=(1, 64, ?, 23), dtype=float32)
Tensor("re_lu_27/Relu:0", shape=(1, 64, ?, 23), dtype=float32)
Tensor("re_lu_29/Relu:0", shape=(1, 128, ?, 23), dtype=float32)
Tensor("re_lu_31/Relu:0", shape=(1, 128, ?, 23), dtype=float32)
Tensor("re_lu_33/Relu:0", shape=(1, 128, ?, 23), dtype=float32)
Tensor("re_lu_35/Relu:0", shape=(1, 256, ?, 23), dtype=float32)
Tensor("re_lu_37/Relu:0", shape=(1, 256, ?, 23), dtype=float32)
Tensor("re_lu_39/Relu:0", shape=(1, 256, ?, 23), dtype=float32)
Tensor("Mean_

## Minimize loss

In [0]:
y_pred = tf.nn.softmax(logits=logits)
y_pred_cls = tf.argmax(y_pred, dimension=1)

y_true_cls = tf.argmax(y_train, dimension=1)

# -------------
#    LOSS
# --------------
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_train)
loss = tf.reduce_mean(cross_entropy)

# -------------
#   ACCURACY
# --------------
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# -------------
# MINIMIZE LOSS
# --------------
train_op = tf.train.AdamOptimizer().minimize(loss)

# 3, Run

In [52]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
#   sess.run(iteration.initializer, feed_dict={x: train_data[0], y: train_data[1]})
  print("Training.....")
  for i in range(NUM_EPOCHS):
    tot_loss = 0
    tot_acc = 0
    for j in range(N_BATCHES):
      idx = np.random.randint(0, num_samples - BATCH_SIZE)
      feed_dict = {x_train: X_train[idx: idx + BATCH_SIZE],
                  y_train: Y_train[idx: idx + BATCH_SIZE]}
    
      _, loss_value, acc_value = sess.run([train_op, loss, accuracy], feed_dict=feed_dict)
      tot_loss += loss_value
      tot_acc += acc_value
    print("Iter: {}, Loss: {:.4f}, Acc: {:.3f}".format(i, tot_loss/ N_BATCHES, tot_acc/ N_BATCHES))

Training.....
Iter: 0, Loss: 3.5479, Acc: 0.148
Iter: 1, Loss: 3.1884, Acc: 0.205
Iter: 2, Loss: 2.9044, Acc: 0.281
Iter: 3, Loss: 2.0713, Acc: 0.462
Iter: 4, Loss: 1.6209, Acc: 0.543
Iter: 5, Loss: 1.5816, Acc: 0.557
Iter: 6, Loss: 1.5063, Acc: 0.524
Iter: 7, Loss: 0.9381, Acc: 0.743
Iter: 8, Loss: 0.9205, Acc: 0.719
Iter: 9, Loss: 0.6939, Acc: 0.776
Iter: 10, Loss: 0.5743, Acc: 0.824
Iter: 11, Loss: 0.4435, Acc: 0.862
Iter: 12, Loss: 0.2572, Acc: 0.905
Iter: 13, Loss: 0.5399, Acc: 0.843
Iter: 14, Loss: 0.3404, Acc: 0.886
Iter: 15, Loss: 0.4969, Acc: 0.876
Iter: 16, Loss: 0.2966, Acc: 0.933
Iter: 17, Loss: 0.3038, Acc: 0.905
Iter: 18, Loss: 0.3387, Acc: 0.910
Iter: 19, Loss: 0.3284, Acc: 0.900
